[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datarobot-community/custom-models/blob/master/drum_overview/Main_Script.ipynb)

## Using MLOps DRUM to test your custom models

**Original Author**: Tim Whittaker

**Modified by**: Mandie Quartly

#### Scope
We'll get our hands dirty by:

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring



In [1]:
#Clone the repository
!git clone https://github.com/datarobot-community/mlops-examples

Cloning into 'mlops-examples'...
remote: Enumerating objects: 1407, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1407 (delta 142), reused 137 (delta 137), pack-reused 1250
Receiving objects: 100% (1407/1407), 110.29 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (637/637), done.
Checking out files: 100% (196/196), done.


In [ ]:
# !pip install -r ./mlops-examples/drum_overview/requirements.txt

%pip install datarobot-drum

## Train a regression model

A simple RandomForestRegressor to predict the compressive strength of high-performance concrete using the Concrete compressive strength dataset found in this paper:

`I-Cheng Yeh, "Modeling of strength of high performance concrete using artificial neural networks" Cement and Concrete Research, Vol. 28, No. 12, pp. 1797-1808 (1998).`.

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

# Read the train and test data
TRAIN_DATA_REG = "./mlops-examples/drum_overview/data/concrete_train.csv"  # 14 features
TEST_DATA_REG = "./mlops-examples/drum_overview/data/concrete_test.csv"  # 13 features - target is removed

reg_X_train = pd.read_csv(TRAIN_DATA_REG)
reg_Y_train = reg_X_train.pop('concrete_compressive_strength')

reg_X_test = pd.read_csv(TEST_DATA_REG)

# Fit the model
rf = RandomForestRegressor()
rf.fit(reg_X_train, reg_Y_train)

# Pickle the file and write it to the file system
with open("./mlops-examples/drum_overview/custom_model_reg/reg_rf_model.pkl", 'wb') as pkl:
    pickle.dump(rf, pkl)
    
# Call predict to confirm it works
rf.predict(reg_X_test)[0:10]

array([49.35179576, 16.05376805, 29.9966807 , 11.05407913,  4.31469944,
       24.99589748, 41.44826343,  5.22090533, 34.01369494, 36.18736849])

## Testing the Model

Input the prediction dataset that includes all features except the target feature.

In [5]:
%%sh 
drum perf-test --code-dir ./mlops-examples/drum_overview/custom_model_reg \
--input ./mlops-examples/drum_overview/data/concrete_test.csv  --target-type regression

DRUM performance test
Model:      /content/mlops-examples/drum_overview/custom_model_reg
Data:       /content/mlops-examples/drum_overview/data/concrete_test.csv
# Features: 8
Preparing test data...



Running test case with timeout: 600
Running test case: 41 bytes - 1 samples, 100 iterations
Running test case with timeout: 600
Running test case: 0.1MB - 2529 samples, 50 iterations
Running test case with timeout: 600
Running test case: 10MB - 252956 samples, 5 iterations
Running test case with timeout: 600
Running test case: 50MB - 1264781 samples, 1 iterations
Test is done, stopping drum server

  size     samples   iters    min     avg     max     total     used     total p
                                                       (s)      (MB)     hysical
                                                                          (MB)  
41 bytes         1     100   0.017   0.018   0.022     1.776        NA        NA
0.1MB         2529      50   0.054   0.057   0.083     2.834        NA  

tput: terminal attributes: No such device or address



## Validating the Model

In [6]:
%%sh 
drum validation --code-dir ./mlops-examples/drum_overview/custom_model_reg \
--input ./mlops-examples/drum_overview/data/concrete_test.csv \
--target-type regression > drum_validation.log

In [7]:
%%sh
tail drum_validation.log



Validation checks results
      Test case          Status   Details
Basic batch prediction   PASSED          
Null value imputation    PASSED          


# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports natively, but for those which DRUM doesn't support off the shelf, we'll just need to create some custom hooks. In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [8]:
%%sh 
drum score --code-dir ./mlops-examples/drum_overview/custom_model_reg \
--input ./mlops-examples/drum_overview/data/concrete_test.csv \
--output ./mlops-examples/drum_overview/data/predictions.csv --target-type regression

In [9]:
pd.read_csv("./mlops-examples/drum_overview/data/predictions.csv").head()

,Predictions
0,49.351796
1,16.053768
2,29.996681
3,11.054079
4,4.314699


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [10]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [11]:
run_inference_server = ["drum",
              "server",
              "--code-dir","./mlops-examples/drum_overview/custom_model_reg", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose"
              ]

In [12]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [13]:
## confirm the server is running
time.sleep(10) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789").content

check status


b'{"message":"OK"}\n'

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [14]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [15]:
# %%timeit
scoring_data = pd.read_csv("./mlops-examples/drum_overview/data/concrete_test.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

{'predictions': [49.3517957612,
                 16.0537680487,
                 29.9966806992,
                 11.0540791281,
                 4.3146994394,
                 24.9958974817,
                 41.4482634256,
                 5.2209053253,
                 34.0136949358,
                 36.1873684923,
                 50.8947809964,
                 20.0421857757,
                 9.6950116159,
                 9.480639738,
                 40.1518692818,
                 9.7036231712,
                 6.1237465682,
                 10.056552146,
                 44.0038475882,
                 47.1183072068,
                 12.461933962,
                 16.6101318422,
                 27.8160542698,
                 36.0255622646,
                 38.4931692896,
                 14.2099417805,
                 16.896988332,
                 31.9657138391,
                 38.0365569138,
                 39.4245134704,
                 14.4808024271,
                 1

In [16]:
requests.request("GET", "http://0.0.0.0:6789/").content

b'{"message":"OK"}\n'

In [17]:
inference_server.terminate()
inference_server.stdout.readlines()

[b'Detected REST server mode - this is an advanced option\n',
 b'Detected /content/mlops-examples/drum_overview/custom_model_reg/custom.py .. trying to load hooks\n',
 b'\x1b \x1b\n',
 b'\x1b \x1b\n',
 b'\x1b============================================================\x1b\n',
 b'\x1bComponent: Prediction Server\x1b\n',
 b'\x1bOutput:\x1b\n',
 b'\x1b------------------------------------------------------------\x1b\n']

In [18]:
#Stop the flask server
# %%sh fuser -n tcp -k 6789
! lsof -i tcp:6789 | grep -v PID | awk '{print $2}' | xargs kill

## Value Prop

One may ask, what is the benefit to be had here?  Well, first off, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit.

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!